# Computational Theory

**Ethan Conneely (G00393941)**


Imports used for countdown

`random` was used for generating random games of countdown for the solver to solve.

`itertools` was used for getting the permutations for all the possible combinations of operations on the numbers.


In [26]:
import random
import itertools

# Game Rules

Here I have defined the rules for a valid game of countdown it can be used to generate a random game to test the solver on it.

The rules for the game are as follows:

- The player may choose the number of large numbers (25,50,75,100) to be used in the game 0-4.
  - The game then picks the number of large number from that list there will not be duplicates
- The remaining number are picked from a selection of 20 number from 1-10 each number is double up in the selection


In [27]:
def generate_game(bigNumbers) -> tuple[list[int], int]:
    if bigNumbers > 4 or bigNumbers < 0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers = random.sample([25, 50, 75, 100], bigNumbers)
    smallNumbers = random.sample(
        list(range(1, 11)) + list(range(1, 11)), 6 - bigNumbers
    )

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))

Here is a lookup for the operations that can be done division is integer division as we cannot at any point in the game have a decimal number


In [28]:
ops = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}

In [29]:
def recusive_solve_countdown(numbers, target, calculations=[]):
    solutions: list = []

    # iterate over all pair of number pairs
    for pair in itertools.combinations(numbers, 2):
        # Remove the pair from the list of numbers and make copy
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        for op, calculate in ops.items():
            a = max(pair)
            b = min(pair)

            if op == "/" and (a == 0 or b == 0):
                continue  # skip division by zero

            if op == "/" and b != 0 and a % b != 0:
                continue  # skip none whole number division

            if op == "/" and b == 1:
                continue  # skip pointeless division

            if op == "*" and b == 1:
                continue  # skip pointeless multiplication

            # if (b == 1):
            #     continue  # skip none whole number division

            global computation_count  # reference the global variable for keeping track of the number of calculations
            computation_count += 1

            val = calculate(a, b)

            calcs = [*calculations, (a, op, b, val)]

            if val == target:
                solutions.append(calcs)

            sols = recusive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions = [*sols, *solutions]

    return solutions

In [30]:
def operations_to_expressions(expr, operations):
    left = operations.get(expr[0])
    if left is None:
        left = expr[0]
    else:
        del operations[expr[0]]
        left = operations_to_expressions(left, operations)

    right = operations.get(expr[2])
    if right is None:
        right = expr[2]
    else:
        del operations[expr[2]]
        right = operations_to_expressions(right, operations)

    return f"({left}{expr[1]}{right})"


`get_expression_lookup` converts the operations that were done to its computed value e.g

```
1+2=3
```

So it would store it in the dictionary as key `3` value `1,+,3` this is used for quick lookup later when recursivly descending the expression tree.

In [31]:
def get_expression_lookup(operations):
    expressions = {}
    for expr in operations:
        expressions[expr[3]] = (expr[0], expr[1], expr[2])
    return expressions

In [32]:
def print_solutions(solutions):

    unique_expressions = []

    for operations in solutions:

        expressions = get_expression_lookup(operations)

        expr_paren = operations_to_expressions(operations[-1], expressions)

        # remove solutions that result in the same expression
        if expr_paren in unique_expressions:
            continue

        unique_expressions.append(expr_paren)

        for expr in operations:
            print(f"{expr[0]}{expr[1]}{expr[2]} = {expr[3]}")

        res = float(eval(expr_paren))

        print(expr_paren + " = " + str(res))
        assert res == float(
            operations[-1][3]
        )  # assert that the evaluation is correct helps catch error when making modifications
        print()

    print("Unique expressions: ", len(unique_expressions))

    print("Found expressions: ", len(solutions))

In [33]:
computation_count = 0

## Fixed Game


In [34]:
numbers = [75, 50, 2, 3, 7, 1]
target = 103


computation_count = 0

solutions = recusive_solve_countdown(numbers, target)

print_solutions(solutions)

print(f"Computations: {computation_count:,}")

7-1 = 6
50-3 = 47
75*2 = 150
150-47 = 103
((75*2)-(50-3)) = 103.0

7-1 = 6
50+3 = 53
75*2 = 150
53-6 = 47
150-47 = 103
((75*2)-((50+3)-(7-1))) = 103.0

7-1 = 6
50+3 = 53
75*2 = 150
150+6 = 156
156-53 = 103
(((75*2)+(7-1))-(50+3)) = 103.0

7-1 = 6
50+3 = 53
75*2 = 150
150-53 = 97
97+6 = 103
(((75*2)-(50+3))+(7-1)) = 103.0

7-1 = 6
50/2 = 25
75-3 = 72
25+6 = 31
72+31 = 103
((75-3)+((50/2)+(7-1))) = 103.0

7-1 = 6
50/2 = 25
75-3 = 72
72+6 = 78
78+25 = 103
(((75-3)+(7-1))+(50/2)) = 103.0

7-1 = 6
50/2 = 25
75-3 = 72
72+25 = 97
97+6 = 103
(((75-3)+(50/2))+(7-1)) = 103.0

7-1 = 6
50/2 = 25
75+3 = 78
78+25 = 103
((75+3)+(50/2)) = 103.0

7-1 = 6
50/2 = 25
6-3 = 3
75+25 = 100
100+3 = 103
((75+(50/2))+((7-1)-3)) = 103.0

7-1 = 6
50/2 = 25
6-3 = 3
75+3 = 78
78+25 = 103
((75+((7-1)-3))+(50/2)) = 103.0

7-1 = 6
50/2 = 25
6-3 = 3
25+3 = 28
75+28 = 103
(75+((50/2)+((7-1)-3))) = 103.0

7-1 = 6
50/2 = 25
75+6 = 81
25-3 = 22
81+22 = 103
((75+(7-1))+((50/2)-3)) = 103.0

7-1 = 6
50/2 = 25
75+6 = 81
81-3 =

## Single Solution


In [35]:
numbers = [25, 50, 5, 9, 3, 10]
target = 899


computation_count = 0

solutions = recusive_solve_countdown(numbers, target)

print_solutions(solutions)

print(f"Computations: {computation_count:,}")

50*9 = 450
450-3 = 447
447*10 = 4470
4470+25 = 4495
4495/5 = 899
(((((50*9)-3)*10)+25)/5) = 899.0

Unique expressions:  1
Found expressions:  1
Computations: 1,375,560


## Random Game


In [36]:
(numbers, target) = generate_game(3)

print("numbers", numbers)
print("target", target)
print()

computation_count = 0
solutions = recusive_solve_countdown(numbers, target)
print_solutions(solutions)
print(f"Computations: {computation_count:,}")

numbers [25, 100, 75, 8, 6, 5]
target 540

6*5 = 30
100+8 = 108
30-25 = 5
108*5 = 540
((100+8)*((6*5)-25)) = 540.0

6*5 = 30
100*75 = 7500
30*8 = 240
7500/25 = 300
300+240 = 540
(((100*75)/25)+((6*5)*8)) = 540.0

6*5 = 30
75/25 = 3
30*8 = 240
100*3 = 300
300+240 = 540
((100*(75/25))+((6*5)*8)) = 540.0

6*5 = 30
75-25 = 50
50+30 = 80
80*8 = 640
640-100 = 540
((((75-25)+(6*5))*8)-100) = 540.0

6*5 = 30
100/25 = 4
30*8 = 240
75*4 = 300
300+240 = 540
((75*(100/25))+((6*5)*8)) = 540.0

6*5 = 30
100/25 = 4
75-30 = 45
8+4 = 12
45*12 = 540
((75-(6*5))*(8+(100/25))) = 540.0

6*5 = 30
75+30 = 105
105-25 = 80
80*8 = 640
640-100 = 540
((((75+(6*5))-25)*8)-100) = 540.0

6*5 = 30
30-25 = 5
75+5 = 80
80*8 = 640
640-100 = 540
(((75+((6*5)-25))*8)-100) = 540.0

6*5 = 30
30+25 = 55
55*8 = 440
440+100 = 540
((((6*5)+25)*8)+100) = 540.0

8*5 = 40
75+6 = 81
40-25 = 15
100*81 = 8100
8100/15 = 540
((100*(75+6))/((8*5)-25)) = 540.0

8*5 = 40
100*6 = 600
75+25 = 100
600+40 = 640
640-100 = 540
((((75+25)*6)+(8*